This Notebook helps in creating news data set(news summary, Reference headline, quality similar headline) by scrapping news article from Inshorts.



*   First it fetches around 30K news article and store their headline and respective URL to drive in csv formt

*   Secondly for every fetched news article it find similar/related new articles



In [ ]:
'''
mount google drive folder
'''
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
'''
import required packages
'''
import requests
import re
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

In [ ]:
'''
define desktop user-agent

USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
headers = {"user-agent" : USER_AGENT}
'''


'\ndefine desktop user-agent\n\nUSER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"\nheaders = {"user-agent" : USER_AGENT}\n'

In [ ]:
'''
pandas configuration for showing complete record
'''
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', None)

In [ ]:
'''
utilities function for retrieving headline, author, publish-date, content, read-more-source for a given news article
'''

def get_article_headline(article_soup):
    article_headline='None'
    div = article_soup.find('<span itemprop="headline">')
    div = article_soup.find('span', attrs = {'itemprop' : 'headline'})
    #print(div)
    article_headline=div.get_text()
    return article_headline.strip()

def get_article_author(article_soup):
    article_author='None'
    span = article_soup.find("span", class_="get_article_author")#<span class="date">
    ##print(div)
    if span!=None:
      article_author=span.get_text()
    return article_author.strip()

def get_publish_date1(article_soup):
    publish_date='None'
    spans = article_soup.findAll("span", class_="fl")#<span class="fl">
    for span in spans:
        text=span.get_text()
        if "Publish Date" in text:
            publish_date=text
    return publish_date.strip()

def get_publish_date(article_soup):
    article_author='None'
    span = article_soup.find("span", class_="time")#<span class="date">
    ##print(div)
    if span!=None:
      article_author=span.attrs['content'].split('T0')[0]
    return article_author.strip()

def get_article_content(article_soup):
    article_content='None'
    div = article_soup.find("div",attrs = {'itemprop' : 'articleBody'})
    article_content=div.get_text()
    return article_content.strip()

def ger_read_more_source(article_soup):
  read_more_source=None
  try:
    read_more_source=article_soup.find("a",attrs = {'class' : 'source'}).text
  except:
    read_more_source='read more source not available'
  return read_more_source

In [ ]:
'''
Utilities function for retrieving similar news article for a given news headline.
This involves 
1. making a google search with given headline as search query
2. retreiving URL's from google search resonses
3. preparing similar news-article URL's by removing search reponse URL's if it is coming from defined  
   websites list('wikipedia', 'facebook', 'youtube' etc.) or URL represent a PDF file
'''
def get_similar_news_urls(article_headline):
  news_urls=[]
  search_url = 'https://www.google.com/search?client=ubuntu&channel=fs&q={}&ie=utf-8&oe=utf-8'.format(article_headline)
  response = requests.get(search_url)
  #print(response.status_code)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html")#html.parser
    #print(response.text)
    search_divs=soup.findAll("div", class_="kCrYT")
    #search_divs=soup.findAll("div", attrs = {'class' : 'kCrYT'})
    #print(len(search_divs))
    for search_div in search_divs:
      has_elem_a=search_div.find('a',href=True)
      if (has_elem_a != None):
        href=has_elem_a['href']
        if '/url?q=' not in href:
          continue
        news_url=href.split('&sa')[0].split('?q=')[1]
        #print('inside if: '+news_url)
        news_urls.append(news_url)
  return news_urls

def is_news_url_has_low_signf(news_url):
  is_news_url_has_no_sign=False
  url_list_min_significance=['https://en.wikipedia.org/','https://www.youtube.com/','https://ms-my.facebook.com/','https://twitter.com','https://www.sportbible.com/','.pdf','.docx','.ppt']
  for blocked_url in url_list_min_significance:
    if blocked_url in news_url:
      is_news_url_has_no_sign=True
  return is_news_url_has_no_sign

def get_similar_headlines_and_urls(article_headline):
  headline_url_pairs=[]
  news_urls=get_similar_news_urls(article_headline)#get similar new article URL
  #print(news_urls)
  for news_url in news_urls:
    if(not is_news_url_has_low_signf(news_url)):
      #print(news_url)
      try: 
        response = requests.get(news_url,timeout=5)
      except:#to take care timeout
        continue
      #print('response.status_code: ',response.status_code)
      if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        try:  
          headline=soup.title.text
          #print(headline)
        except:
          #print('inside except')
          continue
        if '|' in headline:
          headline=headline.split('|')[0].strip()
        elif '- ' in headline:
          headline=headline.split('- ')[0].strip()   
        #just to discard main headline which we have searched
        if (headline != article_headline):
          headline_url_pairs.append((headline,news_url))
  return headline_url_pairs

In [ ]:
'''
provide header json required while making post request
'''
def get_headers():
    return {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "en-IN,en-US;q=0.9,en;q=0.8",
        "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "cookie": "_ga=GA1.2.474379061.1548476083; _gid=GA1.2.251903072.1548476083; __gads=ID=17fd29a6d34048fc:T=1548476085:S=ALNI_MaRiLYBFlMfKNMAtiW0J3b_o0XGxw",
        "origin": "https://inshorts.com",
        "referer": "https://inshorts.com/en/read/",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
        "x-requested-with": "XMLHttpRequest"
    }

'''
utility function for fetching url's and headline from the html response text
'''
def get_all_headlines_and_urls(response_text,base_url):
  soup = BeautifulSoup(response_text, 'lxml')
  headlines = soup.find_all(attrs={"itemprop": "headline"})# <span itemprop="headline">
  headlines=[headline.text for headline in headlines]
  headline_urls = soup.findAll("a",{"class": "clickable"},href=True)#[<a class="clickable"]
  urls=[]
  for headline_url_tag in headline_urls:
    headline_url=base_url+headline_url_tag.attrs['href']
    urls.append(headline_url)
  return (headlines,urls)

In [ ]:
#for testing function 'get_all_headlines_and_urls'
url='https://inshorts.com/en/read/politics' # URL to hit to get list of headline 
base_url='https://inshorts.com' # will be used for formulating an url of a news story after scrapping partial headline
response = requests.get(url)
headlines,urls=get_all_headlines_and_urls(response.text,base_url) # this function call will just provide 25 news-story headline
for headline,url in zip(headlines,urls):
  print(headline,'\n')
  print(url,'\n')

Why are you in politics, go home & cook: BJP leader to Supriya Sule; her husband reacts 

https://inshorts.com/en/news/why-are-you-in-politics-go-home-cook-bjp-leader-to-supriya-sule-her-husband-reacts-1653553445106 

Harsh Vardhan walks out of Delhi L-G's oath ceremony, says 'They didn't keep seats for MPs' 

https://inshorts.com/en/news/harsh-vardhan-walks-out-of-delhi-lgs-oath-ceremony-says-they-didnt-keep-seats-for-mps-1653556123559 

Samajwadi Party names RLD chief Jayant Chaudhary for Rajya Sabha polls 

https://inshorts.com/en/news/samajwadi-party-names-rld-chief-jayant-chaudhary-for-rajya-sabha-polls-1653544036135 

Akhilesh asks Maurya 'You take father's money to build roads?', UP CM says 'Not right' 

https://inshorts.com/en/news/akhilesh-asks-maurya-you-take-fathers-money-to-build-roads-up-cm-says-not-right-1653550537606 

Congress and NCP are our real enemies, not BJP: Sena's Vikas Gogawale 

https://inshorts.com/en/news/congress-and-ncp-are-our-real-enemies-not-bjp-senas-v

In [ ]:
'''
fetch news haedlines and url's by making multiple hhtp requests 
'''
base_url = 'https://inshorts.com' # will be used for formulating an url of a news story after scrapping partial headline
url='https://inshorts.com/en/read/politics' # URL to hit to get list of headline
response = requests.get(url)
headlines,urls=get_all_headlines_and_urls(response.text,base_url)

# get more news
url = 'https://inshorts.com/en/ajax/more_news'
news_offset = "fhalugpk-1"#ml81qvwy-1, apwuhnrm-1
index=0
while True:
  index=index+1
  print('index count: ',index)
  response = requests.post(url, data={"category": "", "news_offset": news_offset}, headers=get_headers())
  if response.status_code != 200:
    print('!!!status code is not 200')
    print(response.status_code)
    break
  response_json = json.loads(response.text)
  news_offset = response_json["min_news_id"]
  html_content_in_response=response_json["html"]
  headlines_temp,urls_temp=get_all_headlines_and_urls(html_content_in_response,base_url)
  '''
    for headline,url in zip(headlines,urls):
      print(headline,'\n')
      print(url,'\n')
    '''
  headlines=headlines+headlines_temp
  urls=urls+urls_temp
headline_url_map={'headlines':headlines,'urls':urls}
headline_url_df=pd.DataFrame(headline_url_map)
headline_url_df.head()

index count:  1
index count:  2
index count:  3
index count:  4
index count:  5
index count:  6
index count:  7
index count:  8
index count:  9
index count:  10
index count:  11
index count:  12
index count:  13
index count:  14
index count:  15
index count:  16
index count:  17
index count:  18
index count:  19
index count:  20
index count:  21
index count:  22
index count:  23
index count:  24
index count:  25
index count:  26
index count:  27
index count:  28
index count:  29
index count:  30
index count:  31
index count:  32
index count:  33
index count:  34
index count:  35
index count:  36
index count:  37
index count:  38
index count:  39
index count:  40
index count:  41
index count:  42
index count:  43
index count:  44
index count:  45
index count:  46
index count:  47
index count:  48
index count:  49
index count:  50
index count:  51
index count:  52
index count:  53
index count:  54
index count:  55
index count:  56
index count:  57
index count:  58
index count:  59
index 

,headlines,urls
0,"Why are you in politics, go home & cook: BJP leader to Supriya Sule; her husband reacts",https://inshorts.com/en/news/why-are-you-in-politics-go-home-cook-bjp-leader-to-supriya-sule-her-husband-reacts-1653553445106
1,"Harsh Vardhan walks out of Delhi L-G's oath ceremony, says 'They didn't keep seats for MPs'",https://inshorts.com/en/news/harsh-vardhan-walks-out-of-delhi-lgs-oath-ceremony-says-they-didnt-keep-seats-for-mps-1653556123559
2,Samajwadi Party names RLD chief Jayant Chaudhary for Rajya Sabha polls,https://inshorts.com/en/news/samajwadi-party-names-rld-chief-jayant-chaudhary-for-rajya-sabha-polls-1653544036135
3,"Akhilesh asks Maurya 'You take father's money to build roads?', UP CM says 'Not right'",https://inshorts.com/en/news/akhilesh-asks-maurya-you-take-fathers-money-to-build-roads-up-cm-says-not-right-1653550537606
4,"Congress and NCP are our real enemies, not BJP: Sena's Vikas Gogawale",https://inshorts.com/en/news/congress-and-ncp-are-our-real-enemies-not-bjp-senas-vikas-gogawale-1653553890132


In [ ]:
headline_url_df.shape

(9002, 2)

In [ ]:
'''
save the headline and news url's  to file and these url's will be used to further find the similar headlines 
'''
headline_url_df.to_csv('/content/headline_url.csv', index=False,header=True) 

In [ ]:
headline_url_df=pd.read_csv('/content/headline_url.csv')

In [ ]:
headline_url_df.shape

(9002, 2)

In [ ]:
print('url_headline dataframe shape: ',headline_url_df.shape)
print('columns in dataframe: ',headline_url_df.columns)
display(headline_url_df[154:155]['urls'])

url_headline dataframe shape:  (9002, 2)
columns in dataframe:  Index(['headlines', 'urls'], dtype='object')


154    https://inshorts.com/en/news/5435-issues-related-to-potholes-bad-roads-revealed-in-bengaluru-1615027193035
Name: urls, dtype: object

In [ ]:
'''
Fetch Headline,content,author,published_date,read_more_source,similar_headline,similar_headline_url from news URL's fetched in previous step.
since this involves programmatically making google search with headline as search query we did it in batches of 50 search 
with some time gap of 6 hours as google was blocking our search requests
'''
#article_url,Headline,content,author,published_date,read_more_source,similar_headline,similar_headline_url
article_urls=[]
headlines=[]
contents=[]
authors=[]
published_dates=[]
read_more_sources=[]
similar_headlines=[]
similar_headline_urls=[]
#counter=10
for index in range(3000,3050,1):
  '''
  if(counter==0):
    counter=10
    time.sleep(60)
  counter=counter-1
  '''
  article_url=headline_url_df['urls'][index]
  response = requests.get(article_url)
  if response.status_code == 200:
    article_soup = BeautifulSoup(response.text, "html")#html.parser
    headline=get_article_headline(article_soup)
    content=get_article_content(article_soup)
    author=get_article_author(article_soup)
    published_date=get_publish_date(article_soup)
    read_more_source=ger_read_more_source(article_soup)
    print('headline: ',headline)
    similar_headline_url_pairs=get_similar_headlines_and_urls(headline)
    similar_headline=[headline_url[0] for headline_url in similar_headline_url_pairs]
    similar_headline_url=[headline_url[1] for headline_url in similar_headline_url_pairs]
    article_urls.append(article_url)
    headlines.append(headline)
    contents.append(content)
    authors.append(author)
    published_dates.append(published_date)
    read_more_sources.append(read_more_source)
    similar_headlines.append(similar_headline)
    similar_headline_urls.append(similar_headline_url)
article_details_dict={'article_url':article_urls,'headline':headlines,'content':contents,'author':authors,'published_date':published_dates,'read_more_source':read_more_sources,'similar_headline':similar_headlines,'similar_headline_url':similar_headline_urls}
#print(article_details_dict)
article_details_df=pd.DataFrame(article_details_dict)
article_details_df.head()


headline:  Vice President Venkaiah Naidu asks all MPs to promote native languages
headline:  Domestic air traffic now closer to pre-COVID-19 levels: Aviation Minister
headline:  K'taka makes COVID-19 negative report mandatory for those arriving from Maha
headline:  Troll asks Shekhar source of income after he shares pics of home, actor responds
headline:  India gives 1 lakh additional COVID-19 vaccine doses to Maldives
headline:  I don't think Smith will stay 11 weeks away from wife for ₹2.2 cr: Clarke on IPL
headline:  PFI takes out rally in Kerala with people dressed like RSS cadres
headline:  Ram Mandir Trust must give accounts of temple donations: Siddaramaiah
headline:  Pics show colour of Alaknanda river before and after flash floods in U'khand
headline:  If highlighting farmers’ protest is sedition, I'm better in jail: Disha in court
headline:  Delhi court reserves order on Disha Ravi's bail plea for Tuesday
headline:  Air India Express flight hits electric pole while landing in

,article_url,headline,content,author,published_date,read_more_source,similar_headline,similar_headline_url
0,https://inshorts.com/en/news/vice-president-venkaiah-naidu-asks-all-mps-to-promote-native-languages-1613832605204,Vice President Venkaiah Naidu asks all MPs to promote native languages,"Vice President M Venkaiah Naidu has written a letter to all MPs, requesting them to contribute towards the promotion of Indian native languages. He said that the first learned and spoken mother tongue is the ""soul of life"". ""Extinction of a language results in the loss of precious legacy. We can't allow this to happen,"" Naidu added.",None,2021-02-20T14:50:05.000Z,read more source not available,"[‘Soul of life’: VP Venkaiah Naidu urges MPs to promote mother tongue, 'Mother tongue is soul of life' — Venkaiah Naidu urges MPs to promote native languages, Vice President asks MPs to promote mother tongue, the soul of life, ‘King Can Do No Wrong’: Venkaiah Naidu Asks Rajya Sabha MPs to Speak in Hindi, VP Venkaiah Naidu calls for promotion, preservation of India's linguistic heritage, Vice President calls for according equal respect to all Indian languages, Venkaiah Naidu for ‘people’s movement’ to protect Telugu]","[https://indianexpress.com/article/india/soul-of-life-vp-venkaiah-naidu-urges-mps-to-promote-mother-tongue-7197099/, https://theprint.in/india/mother-tongue-is-soul-of-life-venkaiah-naidu-urges-mps-to-promote-native-languages/608645/, https://www.topstory.online/vice-president-asks-mps-to-promote-mother-tongue-the-soul-of-life/, https://www.news18.com/news/india/king-can-do-no-wrong-venkaiah-naidu-asks-rajya-sabha-mps-to-speak-in-hindi-1707267.html, https://www.republicworld.com/india-news/general-news/vp-venkaiah-naidu-calls-for-promotion-preservation-of-indias-linguistic-heritage.html, https://vicepresidentofindia.nic.in/pressrelease/vice-president-calls-according-equal-respect-all-indian-languages, https://www.thehindu.com/news/national/andhra-pradesh/venkaiah-naidu-for-peoples-movement-to-protect-telugu/article35647746.ece]"
1,https://inshorts.com/en/news/domestic-air-traffic-now-closer-to-precovid19-levels-aviation-minister-1613832521866,Domestic air traffic now closer to pre-COVID-19 levels: Aviation Minister,"Union Civil Aviation Minister Hardeep Singh Puri has announced that domestic air traffic is now ""within touching distance"" of pre-COVID-19 levels with 2.9 lakh passengers travelling on 2,360 flights on Friday. He added that government may remove the fare band besides lifting other restrictions with domestic air traffic expected to increase further in the summer.",None,2021-02-20T14:48:41.000Z,Hindustan Times,"[Domestic air traffic to cross pre-Covid levels in next 2 months: Scindia, Domestic Air Passenger Traffic To Surpass Pre-covid Levels In Next Two Months: Scindia, \r\n\t\r\n\tEconomic Impacts of COVID-19 on Civil Aviation\r\n\r\n, \r\n\t\r\n\tCOVID-19 Airport Status\r\n\r\n, Novel Coronavirus (COVID-19) Update, Media Details, Domestic air passenger traffic reaching pre-covid levels: Aviation Minister Scindia, India's domestic air passenger traffic to surpass pre-Covid levels in a year, says Scindia]","[https://www.business-standard.com/article/current-affairs/domestic-air-traffic-to-cross-pre-covid-levels-in-next-2-months-scindia-122022100625_1.html, https://www.livemint.com/news/india/domestic-air-passenger-traffic-to-surpass-pre-covid-levels-in-next-two-months-scindia-11645435075814.html, https://www.icao.int/sustainability/Pages/Economic-Impacts-of-COVID-19.aspx, https://www.icao.int/safety/pages/covid-19-airport-status.aspx, https://www.faa.gov/newsroom/novel-coronavirus-covid-19-update-0, https://www.hardeepsinghpuri.com/single-media/domestic-air-traffic-now-closer-to-precovid19-levels-hardeep-singh-puri, https://telanganatoday.com/domestic-air-passenger-traffic-reaching-pre-covid-levels-aviation-minister-scindia, https://theprint.in/economy/indias-domestic-air-passenger-traffic-to-surpass-pre-covid-levels

In [ ]:
'''
article_headline="Russian MMA fighter left with 'Popeye' biceps on injecting petroleum jelly"
#get_similar_news_urls(headline)
get_similar_headlines_and_urls(headline)
search_url = 'https://www.google.com/search?client=ubuntu&channel=fs&q={}&ie=utf-8&oe=utf-8'.format(article_headline)
response = requests.get(search_url)
print(response.status_code)
if response.status_code == 200:
  soup = BeautifulSoup(response.text, "html")#html.parser
    #print(response.text)
  search_divs=soup.findAll("div", class_="kCrYT")
    #search_divs=soup.findAll("div", attrs = {'class' : 'kCrYT'})
    #print(len(search_divs))
  for search_div in search_divs:
    has_elem_a=search_div.find('a',href=True)
    if (has_elem_a != None):
      href=has_elem_a['href']
      if '/url?q=' not in href:
        continue
      news_url=href.split('&sa')[0].split('?q=')[1]
      print('inside if: '+news_url)
      news_urls.append(news_url)
'''

In [ ]:
#article_details_df.to_csv('/content/gdrive/My Drive/capstone_project/news_url_with_headline/data_5.csv', index=False,header=True) 
print('before saving shape : ',article_details_df.shape)
article_details_df.to_pickle('/content/data_60_50.df')
article_details_df=pd.read_pickle('/content/data_60_50.df')
print('after saving shape : ',article_details_df.shape)
display(article_details_df[['article_url','similar_headline','similar_headline_url']])

before saving shape :  (50, 8)
after saving shape :  (50, 8)


,article_url,similar_headline,similar_headline_url
0,https://inshorts.com/en/news/vice-president-venkaiah-naidu-asks-all-mps-to-promote-native-languages-1613832605204,"[‘Soul of life’: VP Venkaiah Naidu urges MPs to promote mother tongue, 'Mother tongue is soul of life' — Venkaiah Naidu urges MPs to promote native languages, Vice President asks MPs to promote mother tongue, the soul of life, ‘King Can Do No Wrong’: Venkaiah Naidu Asks Rajya Sabha MPs to Speak in Hindi, VP Venkaiah Naidu calls for promotion, preservation of India's linguistic heritage, Vice President calls for according equal respect to all Indian languages, Venkaiah Naidu for ‘people’s movement’ to protect Telugu]","[https://indianexpress.com/article/india/soul-of-life-vp-venkaiah-naidu-urges-mps-to-promote-mother-tongue-7197099/, https://theprint.in/india/mother-tongue-is-soul-of-life-venkaiah-naidu-urges-mps-to-promote-native-languages/608645/, https://www.topstory.online/vice-president-asks-mps-to-promote-mother-tongue-the-soul-of-life/, https://www.news18.com/news/india/king-can-do-no-wrong-venkaiah-naidu-asks-rajya-sabha-mps-to-speak-in-hindi-1707267.html, https://www.republicworld.com/india-news/general-news/vp-venkaiah-naidu-calls-for-promotion-preservation-of-indias-linguistic-heritage.html, https://vicepresidentofindia.nic.in/pressrelease/vice-president-calls-according-equal-respect-all-indian-languages, https://www.thehindu.com/news/national/andhra-pradesh/venkaiah-naidu-for-peoples-movement-to-protect-telugu/article35647746.ece]"
1,https://inshorts.com/en/news/domestic-air-traffic-now-closer-to-precovid19-levels-aviation-minister-1613832521866,"[Domestic air traffic to cross pre-Covid levels in next 2 months: Scindia, Domestic Air Passenger Traffic To Surpass Pre-covid Levels In Next Two Months: Scindia, \r\n\t\r\n\tEconomic Impacts of COVID-19 on Civil Aviation\r\n\r\n, \r\n\t\r\n\tCOVID-19 Airport Status\r\n\r\n, Novel Coronavirus (COVID-19) Update, Media Details, Domestic air passenger traffic reaching pre-covid levels: Aviation Minister Scindia, India's domestic air passenger traffic to surpass pre-Covid levels in a year, says Scindia]","[https://www.business-standard.com/article/current-affairs/domestic-air-traffic-to-cross-pre-covid-levels-in-next-2-months-scindia-122022100625_1.html, https://www.livemint.com/news/india/domestic-air-passenger-traffic-to-surpass-pre-covid-levels-in-next-two-months-scindia-11645435075814.html, https://www.icao.int/sustainability/Pages/Economic-Impacts-of-COVID-19.aspx, https://www.icao.int/safety/pages/covid-19-airport-status.aspx, https://www.faa.gov/newsroom/novel-coronavirus-covid-19-update-0, https://www.hardeepsinghpuri.com/single-media/domestic-air-traffic-now-closer-to-precovid19-levels-hardeep-singh-puri, https://telanganatoday.com/domestic-air-passenger-traffic-reaching-pre-covid-levels-aviation-minister-scindia, https://theprint.in/economy/indias-domestic-air-passenger-traffic-to-surpass-pre-covid-levels-in-a-year-says-scindia/888462/]"
2,https://inshorts.com/en/news/ktaka-makes-covid19-negative-report-mandatory-for-those-arriving-from-maha-1613829572091,"[K'taka makes RT-PCR result or vax certificate mandatory for Maha travelers, K'taka makes RT-PCR test mandatory for entry from Kerala, Maha, Covid-19: Negative RT-PCR certificate not older than 72 hours compulsory for those arriving in Karnataka from Maharashtra, K'taka Seeks Travel Ban on Flyers from Omicron-hit Countries, Makes RT-PCR Test Mandatory For Kerala & Maha, No Need Of -ve Report For Maha Passengers, Maha travellers to K'taka must carry Covid negative report]","[https://www.business-standard.com/article/current-affairs/k-taka-makes-rt-pcr-result-or-vax-certificate-mandatory-for-maha-travelers-121063000124_1.html, https://www.thestatesman.com/india/ktaka-rt-pcr-mandatory-kerala-maha-1502988700.html, https://www.deccanherald.com/state/top-karnataka-stories/covid-19-negative-rt-pcr-certificate-not-older-than-72-hours-compulsory-for-those-arri